In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import tensorflow as tf

2023-11-10 22:28:30.657289: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 22:28:30.699893: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-10 22:28:30.701622: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-10 22:28:31.430193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

data = pd.read_csv("./all_grains_data.csv")
data

,ticker,commodity,date,open,high,low,close,volume
0,ZC=F,Corn,2000-07-17,182.25,182.25,178.25,179.25,19385
1,ZC=F,Corn,2000-07-18,179.00,183.25,179.00,180.75,25205
2,ZC=F,Corn,2000-07-19,180.00,182.75,178.75,182.00,17126
3,ZC=F,Corn,2000-07-20,181.50,187.00,181.00,186.00,18742
4,ZC=F,Corn,2000-07-21,185.50,188.00,185.00,187.75,16814
...,...,...,...,...,...,...,...,...
34854,ZS=F,Soybean,2023-09-05,1359.50,1359.50,1348.00,1349.00,85
34855,ZS=F,Soybean,2023-09-06,1365.50,1365.50,1359.50,1360.25,102
34856,ZS=F,Soybean,2023-09-07,1349.50,1350.00,1345.00,1345.00,310
34857,ZS=F,Soybean,2023-09-08,1339.00,1349.50,1339.00,1349.50,198


In [9]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
sys_details = tf.sysconfig.get_build_info()
cuda = sys_details["cuda_version"]
cudnn = sys_details["cudnn_version"]
print(cuda, cudnn)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


11.8 8
Num GPUs Available:  0


In [4]:
corn_data = data[data['commodity'] == 'Corn']
corn_data = data
#corn_data = data
X= corn_data[['open', 'high', 'low', 'volume']]
y = corn_data['close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
average_corn_price = corn_data['close'].mean()
average_corn_price

573.6948790880854

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30, activation='relu', kernel_initializer="glorot_uniform", input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30, activation='relu', kernel_initializer="glorot_uniform"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30, activation='relu', kernel_initializer="glorot_uniform"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1)
])

In [7]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [8]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)

Epoch 1/100
872/872 [==============================] - 5s 4ms/step - loss: 487213.5625 - val_loss: 413182.3125
Epoch 2/100
872/872 [==============================] - 3s 4ms/step - loss: 315750.9062 - val_loss: 223046.3125
Epoch 3/100
872/872 [==============================] - 3s 4ms/step - loss: 140281.5312 - val_loss: 66390.7188
Epoch 4/100
872/872 [==============================] - 2s 3ms/step - loss: 48013.2930 - val_loss: 10536.3750
Epoch 5/100
872/872 [==============================] - 2s 3ms/step - loss: 17094.2969 - val_loss: 4047.1831
Epoch 6/100
872/872 [==============================] - 2s 3ms/step - loss: 8562.6953 - val_loss: 202.3496
Epoch 7/100
872/872 [==============================] - 2s 3ms/step - loss: 7527.9722 - val_loss: 340.0458
Epoch 8/100
872/872 [==============================] - 2s 3ms/step - loss: 6958.9487 - val_loss: 887.3571
Epoch 9/100
872/872 [==============================] - 2s 2ms/step - loss: 6447.0962 - val_loss: 240.1093
Epoch 10/100
872/872 [=====

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score)

Test loss: 201.80596923828125


In [ ]:
from sklearn.model_selection import KFold

# Número de splits para a validação cruzada
n_splits = 3

# Crie o objeto KFold
kf = KFold(n_splits=n_splits)

# Variável para armazenar as pontuações de cada fold
scores = []

y_train = y_train.reset_index(drop=True)
# Loop sobre cada fold
for train_index, val_index in kf.split(X_train):
    # Divida os dados em treinamento e validação para o fold atual
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Defina o modelo
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(30, activation='relu', kernel_initializer='glorot_uniform', input_shape=(X_train.shape[1],)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(30, activation='relu', kernel_initializer='glorot_uniform'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(30, activation='relu', kernel_initializer='glorot_uniform'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, kernel_initializer='glorot_uniform')
    ])

    # Compile o modelo
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Treine o modelo
    model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold), epochs=100, batch_size=32, verbose=0)

    # Avalie o modelo no conjunto de validação
    score = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    
    # Adicione a pontuação à lista de pontuações
    scores.append(score)

# Imprima a média das pontuações de validação cruzada
print('Cross-validation score: ', np.mean(scores))

KeyboardInterrupt: 